In [35]:
import pandas
from datetime import datetime
import numpy

sales = pandas.read_csv("Data - Hack UPC/sales.csv", index_col=0)

sku = pandas.read_csv("Data - Hack UPC/sku.csv", index_col=0)
geo = pandas.read_csv("Data - Hack UPC/geo_params.csv", index_col=0)

data = sales.join(sku, on="SKU").join(geo, on="geoCluster")
data.date = pandas.to_datetime(data.date)

# All the rows with price = nan have also the sales = nan. Assume they are entries without sales
data.drop(["Units"], axis=1, inplace=True)
data.loc[data["SKU"] == 660399, "lagerUnitQuantity"] = 0.4
data["price"] = data.price.fillna(0)
data["sales"] = data.sales.fillna(0)
data["income"] = data.price * data.sales
data["weekday"] = data.date.dt.day_of_week

data = data.sort_values("date")

data = data[data.date > '2020-04-03']

data.to_csv("data.csv")

# Datasets
## Sales dataset

The rows of this dataset are for a sale of (location, product, day). The ID is computed from this triplet.

- Location is the geoCluster it is created?
- Product is the SKU
- Day is the date

For some sales, no products were sold but an entry appears. This is normal in those datasets. Note that all the entries without a "sale" value do not have a "price" value.

The price is unitary.

  
## SKU dataset

- SKU > index
- category > Almost no repeated values
- Type > almost no repeated values
- brandId > some missing values. 1 brand is repeated 10 times, the next ones are 5, 3, 2,2,2, and 1 the others
- lagerUnitQuantity > Decimals, might be related with decimal sales
- Units > full nans >> DISCARD
- trademark > some nans :/ similar to brandId, some are repeated
- countryOfOrigin > some nans :/
- Group > THIS MIGHT BE USEFUL 

## Geo dataset
This dataset relates geographic clusters with cities identifiers.

There are 3 city identifiers, and 450, 64, and 1 geoClusters respectively (?) REVIEW



# Clean data

The product 660399, which is the same as 591899 but with different largeUnitQuantity. The first one has a lUQ of 400 while the other has a lUQ of 1. We assume this is an error, as it is flour and it makes sense to have packages of 1kg and 400g. Also, when comparing the median price of the (5XXX * 0.4) vs 6XXX we have (32,996 vs 28.49). 

```python
data[(data.price != 0) & (data.SKU == 591899)].price.median() * .4, data[(data.price != 0) & (data.SKU == 660399)].price.median()
```

We change this to 400g (.4)

We also detected that until the 4th of April 2020 we have almost no data, so we started our predictor at this date.